In [12]:
import torch
torch.cuda.device_count()

0

In [13]:
%matplotlib inline
import argparse
import functools
import multiprocessing
import random
from copy import deepcopy
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy import ndimage as ndi
import time

## Functions

### Visualization

In [18]:
def get_bool_mask(image_shape, seam):
    bool_mask = np.ones(shape=image_shape, dtype=np.bool)

    for row, col in seam:
        bool_mask[row, col] = False

    return bool_mask


# https://github.com/andrewdcampbell/seam-carving
def visualize(image, bool_mask=None):
    display = image.astype(np.uint8)

    if bool_mask is not None:
        display[np.where(bool_mask == False)] = np.array([0, 0, 255])
    cv2.imshow("visualization", display)
    cv2.waitKey(100)

    return display


def remove_seams(image, bool_mask, dir):
    rows, cols = image.shape[:2]

    bool_mask = np.stack([bool_mask] * 3, axis=2)

    if dir == 'row':
        image = np.transpose(np.transpose(image)[np.transpose(bool_mask)].reshape(3, cols, rows - n))
    elif dir == 'col':
        image = image[bool_mask].reshape((rows, cols - n, 3))
    else:
        pass
        # alert

    return image

def visualize_seams(image, original_mask=None):
    display = image.astype(np.uint8)

    if original_mask is not None:
        display[np.where(original_mask < 0)] = np.array([255, 0, 0])
    return display


def generate_original_mask(original_shape, dir):
    original_mask = np.zeros(original_shape)

    if dir == 'col':
        for i in range(original_shape[0]):
            original_mask[i, :] = range(original_shape[1])
    else:
        for i in range(original_shape[1]):
            original_mask[:, i] = range(original_shape[0])

    return original_mask.astype(int)


### Energy

In [19]:
def saliency_spectral_residual(img):
    _, saliency_map = cv2.saliency.StaticSaliencySpectralResidual_create().computeSaliency(img)
    return saliency_map/255

def saliency_fine_grained(img):
    _, saliency_map = cv2.saliency.StaticSaliencyFineGrained_create().computeSaliency(img)
    return saliency_map/255

## MIX
def salispec_for(img):
    return 0.5*saliency_spectral_residual(img)+0.5*forward_energy(img)

# https://github.com/andrewdcampbell/seam-carving
def backward_energy(image):
    """
    Simple gradient magnitude energy map.
    """
    xgrad = ndi.convolve1d(image, np.array([1, 0, -1]), axis=1, mode='wrap')
    ygrad = ndi.convolve1d(image, np.array([1, 0, -1]), axis=0, mode='wrap')

    grad_mag = np.sqrt(np.sum(xgrad ** 2, axis=2) + np.sum(ygrad ** 2, axis=2))

    #vis = visualize(grad_mag)
    #cv2.imwrite("backward_energy_demo.jpg", vis)

    return grad_mag / 255.0


# https://github.com/andrewdcampbell/seam-carving
def forward_energy(image):
    """
    Forward energy algorithm as described in "Improved Seam Carving for Video Retargeting"
    by Rubinstein, Shamir, Avidan.
    Vectorized code adapted from
    https://github.com/axu2/improved-seam-carving.
    """
    h, w = image.shape[:2]
    g_image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2GRAY).astype(np.float64)

    energy = np.zeros((h, w))
    m = np.zeros((h, w))

    U = np.roll(g_image, 1, axis=0) #每行向下挪一格
    L = np.roll(g_image, 1, axis=1) #每列向右挪一格
    R = np.roll(g_image, -1, axis=1)#每列向左挪一格

    cU = np.abs(R - L)
    cL = np.abs(U - L) + cU
    cR = np.abs(U - R) + cU

    for i in range(1, h):
        mU = m[i - 1]
        mL = np.roll(mU, 1)
        mR = np.roll(mU, -1)

        mULR = np.array([mU, mL, mR])
        cULR = np.array([cU[i], cL[i], cR[i]])
        mULR += cULR

        argmins = np.argmin(mULR, axis=0)
        m[i] = np.choose(argmins, mULR)
        energy[i] = np.choose(argmins, cULR)

    #vis = visualize(energy)
    #cv2.imwrite("forward_energy_demo.jpg", vis)

    return energy / 255.0

## 根据能量图求出seam的总能量
def fitness(energy_map, dir, individual):
    rows, cols = energy_map.shape[:2]

    seam = create_seam(individual, dir)

    energy = 1.0  # distinguish from the 0 case -- out of bound #增加了对出界的惩罚力度
    if dir == 'row':
        for row, col in seam:
            # determine if it is out of bound
            if row < 0 or row >= rows:
                return 10000.0
            energy += energy_map[row, col]
    elif dir == 'col':
        for row, col in seam:
            if col < 0 or col >= cols:
                return 10000.0
            energy += energy_map[row, col]
    return energy ** np.e

### Init

In [20]:
def create_population(population_size, image_shape, dir):
    return [create_individual(image_shape, dir) for _ in range(population_size)]

def create_individual(image_shape, dir):
    '''
    individual -- pivot_loc, randomly generated path values
    dir:dir_value row:0/col:1
    '''
    path = list(np.random.random_integers(low=-1, high=1, size=image_shape[1-dir])) 
    
    pivot_index = np.random.randint(low=0, high=image_shape[1-dir])
    pivot_value = np.random.randint(low=0, high=image_shape[dir])
    
    ##只有pivot的地方有值，其他地方为【-1，1】
    path[pivot_index] = pivot_value
        
    return pivot_index, path

### Create seam

In [21]:
## 根据individual的pivot和偏移量画出seam
def create_seam(individual, dir):
    pivot, path = individual

    if dir == 'row':
        return [(f(pivot, path, i), i) for i in range(len(path))]
    elif dir == 'col':
        return [(i, f(pivot, path, i)) for i in range(len(path))]
    else:
        pass  # alert


def f(pivot, path, index):
    if index == pivot:
        return path[index]
    elif index > pivot:
        return path[index] + f(pivot, path, index - 1)
    elif index < pivot:
        return path[index] + f(pivot, path, index + 1)

### 进化过程的操作函数

In [22]:
# roulette - "stochastic acceptance"
# https://en.wikipedia.org/wiki/Fitness_proportionate_selection
def select(population, fitness):
    '''
    Calculate the importance of the seams using Engergy function and select the most important seam
    '''
    total = sum(fitness)

    selection_pool = []
    
    # 多了复制操作因此杂交的所需的父母减少reproduction_num个
    while len(selection_pool) < len(population)-reproduction_num:
        index = np.random.randint(low=0, high=len(population))
        fit = fitness[index]

        if fit > 0.0:  # ignore the out-of-bound seam
                        ## (fit / total)  ： importance
                        ## 1.0 - (fit / total)  : unimportance
            probability = 1.0 - (fit / total)  

            if random.random() < probability:  # smaller fitness value -- a larger prob to be selected
                selection_pool.append(population[index])

    return selection_pool


def reproduct(population, fitness):
    '''
    复制：复制能量最小的reproduction_num个seam
    '''
    # 对原population的fitness的从小到大排序
    reproduction_index=np.array(fitness).argsort()
        
    loss=0
    i = 0
    reproduction_pool=[]
    
    for index in reproduction_index:
        if i >= reproduction_num:
            break
        fit = fitness[index]
        if fit > 0.0:
            if i == 0:
                loss=fit 
            reproduction_pool.append(population[index])
            i = i + 1
            
    return reproduction_pool, loss
    
# single point
def cross(individual1, individual2):
    ## pivot_index, path
    pi1, path1 = individual1
    pi2, path2 = individual2

    # keep track of pivot values
    #return removed value and list has been changed
    pv1 = path1.pop(pi1)
    pv2 = path2.pop(pi2)

    ## index
    point = np.random.randint(0, len(path1))
    
    ## 随机对调两个parent的path的一部分（因为是以【-1，1】编码所以不会出现畸变）

    path1[point:], path2[point:] = path2[point:], path1[point:]

    path1.insert(pi1, pv1)
    path2.insert(pi2, pv2)


# some kind of gaussian mutation
def mutate(individual, kernel):
    pivot, path = individual

    size = len(path)
    kernel_size = int(np.ceil(len(kernel) / 2))

    point = np.random.randint(low=0, high=size)
    window = [point + i for i in range(1 - kernel_size, kernel_size)]

    # print("mutate", kernel)

    for i in range(len(window)):
        index = window[i]
        if 0 <= index < size and index != pivot:
            if np.random.random() < kernel[i]:
                path[index] = np.random.randint(low=-1, high=2)


def spread(code, gap):
    '''
    code: ndarray
    gap: int (absolute value)
    return: ndarray
    '''
    indices = [np.random.randint(low=0, high=len(code)) for i in range(gap)]
    for idx in indices:
        code[idx] += -1

    code[np.argwhere(code<-1)] = -1

    return code

def ternary_encode(init_code, val):
    '''
    init_code: list of ones
    val: int
    '''
    code = spread(np.array(init_code), len(init_code)-val)
    while np.min(code) < -1:
        code = spread(code, np.sum(code)-val)

    code = list(code)
    return code


def crossover2(individual1, individual2):
    '''
    implemened by "An Improved Genetic Algorithms-based Seam Carving Method"
    flexible pivot value
    '''
    pi1, path1 = individual1
    pi2, path2 = individual2

    pv1 = path1.pop(pi1)
    pv2 = path2.pop(pi2)

    point1 = np.random.randint(0, len(path1))
    path1[point1:], path2[point1:] = path2[point1:], path1[point1:]

    if pv1 != pv2:
        # pivot ternary encoding
        code_l = [1]*max(pv1, pv2)  # larger
        code_s = deepcopy(code_l)  # smaller
        code_s = ternary_encode(code_s, min(pv1, pv2))

        point2 = np.random.randint(0, len(code_l))
        code_l[point2:], code_s[point2:] = code_s[point2:], code_l[point2:]

        if pv1 > pv2:
            path1.insert(pi1, np.sum(code_l))
            path2.insert(pi2, np.sum(code_s))
        else:
            path1.insert(pi1, np.sum(code_s))
            path2.insert(pi2, np.sum(code_l))
    else:
        path1.insert(pi1, pv1)
        path2.insert(pi2, pv2)
        
def gaussian(size, sigma):
    size = int(np.ceil(size / 2))
    r = range(1 - size, size)
    kernel = []

    for x in r:
        kernel.append(np.exp(-np.power(x, 2) / (2 * np.power(sigma, 2))))

    return kernel

### 进化的主要函数

In [23]:
def new_generation(last_pop, energy_map, dir):
    '''
    一次完整的进化过程： 1. 先通过fitness确认被选择和不变的（复制）seam 2. 进行杂交和突变 3. 结合1和2的seams形成新的种群
    last_pop: 进化使用的群体
    '''
    
    fit_scores = pool.map(functools.partial(fitness, energy_map, dir), last_pop) # map each individual in the population
            
    selection_pool = select(last_pop, fit_scores)
    reproduction_pool, loss = reproduct(last_pop, fit_scores)
    
    # fit_scores = pool.map(functools.partial(fitness, energy_map, dir), reproduction_pool)
    # print("reproduction_fit_scores={}".format(fit_scores))
    
    selection_pool = pool.map(deepcopy, selection_pool)
    reproduction_pool = pool.map(deepcopy, reproduction_pool)

    kernel = gaussian(21, 3.0)
    
    for individual1, individual2 in zip(selection_pool[::2], selection_pool[1::2]):  # sequential, 前后
        crossover2(individual1, individual2)
        mutate(individual1, kernel)
        mutate(individual2, kernel)
    
    selection_pool=selection_pool+reproduction_pool
    
    # fit_scores = pool.map(functools.partial(fitness, energy_map, dir), selection_pool)
    
    # 避免重复 进行shuffle
    random.shuffle(selection_pool)

    return selection_pool, loss


def process_seam(target_image, energy_func, dir):
    '''
    num_generations次进化过程，最后从最终种群中提出一个最佳线路
    determine seams for each step in either direction;
    energy_func: energy function
    dir: dict item
    result: seam
    '''
    
    image_shape = target_image.shape[:2]
    
    # initialize
    ## dir[0]:key[row/col)] dir[1]:value[0/1]
    population = create_population(pop_size, image_shape, dir[1]) 
    energy_map = energy_func(target_image)

    loss=[]   
    
    for generation in range(num_generations):
        population, l = new_generation(population, energy_map, dir[0])
        loss.append(l)

    # determine the solution
    fit_scores = pool.map(functools.partial(fitness, energy_map, dir[0]), population)
    elite = np.argmin(fit_scores)  
    loss.append(fit_scores[elite])
        
    seam = create_seam(population[elite], dir[0])

    return seam, loss



## MAIN

In [14]:
## Hyperparams
pop_size = 10  ## 群体内的个体数量
reproduction_num = 2 ## 复制个数
n = 1 # TODO: 设定一次可以切割多条线
dirs = {'row':0, 'col':1} # 代表横向切割和纵向切割的哈希表
generations = [30] # 迭代次数

In [ ]:
for num_generations in generations:
    
    file_name = "bicycle1" 
    img_path = f"data/{file_name}.png" 
    input_image=cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    target_image = input_image
    
    h_img, w_img, _ = target_image.shape
    target_shape = (int(0.8 * h_img), int(0.75 * w_img)) # 或者也可以设置固定大小（int, int）
    original_shape = target_image.shape[:2]
    ##gap=target_image.shape[0] - target_shape[0] + target_image.shape[1] - target_shape[1]

    # create pool for multiprocessing
    pool = multiprocessing.Pool()
    
    #*************************************************
    print("设定迭代次数为{}".format(num_generations))
    #*************************************************
    
    losslist=[]
    start=time.time()
    
    for item in dirs.items():
        while target_image.shape[item[1]] > target_shape[item[1]]:
            
            diff = target_image.shape[item[1]] - target_shape[item[1]]
            print("Carving {} and the gap to target is **{}**".format(item[0], diff))
            
            # 生成最优路线
            seam, loss = process_seam(target_image, salispec_for, item) 
            
            # 移除目标路线的所在像素
            mask = get_bool_mask(target_image.shape[:2], seam)
            
            # 生成目标图片
            target_image = remove_seams(target_image, mask, item[0])
            
            losslist.append(loss)
            
    end=time.time()    
    print("time={}".format(end-start))

    ##Imgae Save
    cv2.imwrite("result/{}_gen{}_shape{}_{}_time{}.jpg".format(file_name, num_generations, target_shape[0], target_shape[1], end-start), cv2.cvtColor(target_image, cv2.COLOR_RGB2BGR))

    ## Loss save
    ## 确认随着迭代次数的增加，找到的路线的损失也在逐渐变小（优化算法的有效性）
    losslist1=np.array(losslist)
    plt.plot(range(num_generations+1), np.sum(losslist1,axis=0)/gap)
    plt.xlabel("Generation")
    plt.ylabel("Energy")
    plt.savefig("result/{}_loss_gen{}_shape{}_{}_time{}.jpg".format(file_name, num_generations, target_shape[0], target_shape[1], end-start))
    plt.show()